# TDM for IGWBS 1: Daten von der OAI-Schnittstelle von e-rara

## Womit arbeiten wir hier eigentlich?

In [1]:
!python --version

Python 3.8.10


In [2]:
# vorinstallierte Libraries - bei Interesse "#" vor dem Code entfernen und ausführen
#!pip list

In [3]:
# Installation fehlender Libraries
!pip install beautifulsoup4 

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Import der (installierten) Libraries, die im Nachfolgenden benutzt werden

from IPython.display import IFrame              # Webpages in Jupyter Notebook anzeigen/einbetten
import requests                                 # Web-URLs abfragen
from bs4 import BeautifulSoup as soup           # Webscraping und HTML/XML parsen
import lxml                                     # XML Parser für Beautiful Soup
                                                
import os                                       # System und z.B. Ordner navigieren und manipulieren 
import re                                       # mit Regex (Regular Expressions) arbeiten
import pandas as pd                             # Standard Library für Dataframes (tabelleartige Datenstrukturen)
print("Alle Libraries erfolgreich importiert")

Alle Libraries erfolgreich importiert


## Wie sieht die OAI-Schnittstelle im Web aus?

Die OAI-Schnittstelle von e-rara ist frei zugänglich: ES braucht keine Registrierung oder API Key zur Nutzung der Daten. Diese stehen unter [freien Lizenzen](https://www.e-rara.ch/wiki/termsOfUse) zur Verfügung.

Mit`IFrame` können Websites in einem Jupyter Notebook angezeigt/eingebunden werden, so auch die OAI-Startseite von e-rara, `https://www.e-rara.ch/oai?verb=Identify`.

Probieren Sie verschiedene **OAI verbs** in der OAI-URL im Code unten aus:
* `Identify`
* `ListMetadataFormats`
* `ListSets`.

In [5]:
# Identify = Startseite der OAI-Schnittstelle
IFrame('https://www.e-rara.ch/oai?verb=Identify', width=820, height=300)

Mit dem OAI verb `GetRecord` und dem entsprechenden **e-rara-Identifier** lassen sich einfach die Metadaten eines bestimmten Titels anzeigen.

In der URL des Titel auf der e-rara-Plattform, z.B. https://www.e-rara.ch/zut/content/titleinfo/26079348, ist der e-rara-Identifier enthalten, im Beispiel: `26079348`.

Probieren Sie verschiedene Metadatenstandards aus. Diese lassen sich mit dem Parameter `metadataPrefix` in der URL einfach variieren:
* `oai_dc` - [Simple Dublin Core](http://purl.org/dc/elements/1.1/)
* `mods` - [MODS](https://www.loc.gov/standards/mods/)
* `mets`- [METS](https://www.loc.gov/standards/mets/)

Und natürlich können Sie auch andere e-rara-Identifier ausprobieren,z.B. `26079348`!

In [6]:
# Beipieldatensatz aus e-rara
IFrame('https://www.e-rara.ch/oai?verb=GetRecord&metadataPrefix=oai_dc&identifier=20329783', width=820, height=300)

## XML-Daten von der OAI-Schnittstelle beziehen

Bisher haben wir die **Metadaten im  XML-Format** nur auf einer Website angezeigt erhalten - nun greifen wir direkt auf diese Daten zu, d.h. laden sie herunter und speichern sie in Dateien. Hierfür werden zwei kleine Funktionen definiert: `load_xml`und `download_record`. Bei Letzterem wird das OAI verb `GetRecord` benutzt.

In [7]:
# Basis-URL der OAI-Schnittstelle als definierte Variable macht die weitere Verwendung einfacher
oai = 'https://www.e-rara.ch/oai'

In [8]:
# allgemeine Funktion zur Datenabfrage der OAI-Schnittstelle und Dekodierung in XML
def load_xml(params):
    '''
    Accesses the OAI interface according to given parameters and scrapes its content.
    Parameters:
    All available native OAI verbs and parameter/value pairs.
    '''
    base_url = oai
    response = requests.get(base_url, params=params)
    output_soup = soup(response.content, features="xml")   #"lxml"
    return output_soup

In [9]:
# "load_xml"-Funktion benötigt die Parameter, die in den URLs oben angehängt wurden
xml_soup = load_xml({'verb': 'Identify'})
xml_soup

<?xml version="1.0" encoding="utf-8"?>
<OAI-PMH xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/ http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd"><responseDate>2022-06-14T15:22:59Z</responseDate><request verb="Identify">https://www.e-rara.ch/oai/</request><Identify><repositoryName>Visual Library Server</repositoryName><baseURL>https://www.e-rara.ch/oai/</baseURL><protocolVersion>2.0</protocolVersion><adminEmail>issue-erara@library.ethz.ch</adminEmail><earliestDatestamp>2009-11-10T09:38:31Z</earliestDatestamp><deletedRecord>no</deletedRecord><granularity>YYYY-MM-DDThh:mm:ssZ</granularity></Identify></OAI-PMH>

In [10]:
# OAI verb, metadataPrefix, identifier als Parameter für die "load_xml"-Funktion
# entspricht: https://www.e-rara.ch/oai?verb=GetRecord&metadataPrefix=oai_dc&identifier=26079348

xml_soup = load_xml({'verb': 'GetRecord', 'metadataPrefix': 'oai_dc', 'identifier': '26079348'})
xml_soup

<?xml version="1.0" encoding="utf-8"?>
<OAI-PMH xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/ http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd"><responseDate>2022-06-14T15:22:59Z</responseDate><request identifier="26079348" metadataPrefix="oai_dc" verb="GetRecord">https://www.e-rara.ch/oai/</request><GetRecord><record><header><identifier>oai:www.e-rara.ch:26079348</identifier><datestamp>2021-10-27T06:45:24Z</datestamp><setSpec>zut</setSpec><setSpec>book</setSpec></header><metadata><oai_dc:dc xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd"><dc:title>Handbuch der Bibliothekswissenschaft, der Literatur- und Bücherkunde : eine gedrängte Uebersicht der Handschriftenk

In [11]:
# Funktion um Metadaten eines Titels herunterzuladen - in verschiedenenen Metadatenstandards und in genuiner XML-Formatierung

def download_record(ID, metadataPrefix='mods'):
    '''
    Downloads a certain metadata record in original XML formatting from OAI to a single XML file.
    Throws a notice when download is successful and if download fails.
    Parameters:
    ID = E-rara ID of the desired metadata record.
    metadataPrefix = Metadata format to be delivered. Can be: oai_dc, mods, rawmods, mets. Default value is mods.
    '''
    path = os.getcwd()
    output_soup = load_xml({'verb': 'GetRecord', 'metadataPrefix': metadataPrefix, 'identifier': ID})
    outfile = path + '/{}.xml'.format(ID) 
    try:
        with open(outfile, mode='w', encoding='utf-8') as f:
            f.write(output_soup.decode())
            print("Metadata file {}.xml saved".format(ID))
    except:
            print("Saving metadata file failed".format(ID))
    finally:
            pass

In [12]:
# die Funkion in Aktion
download_record('26079348')
#download_record('26079348', metadataPrefix='oai_dc')

Metadata file 26079348.xml saved


In [13]:
# auch mehrere Titel sind mittels einer FOR-Schleife möglich
for ID in ['22400512', '4310610', '13621365']:
    download_record(ID, metadataPrefix='oai_dc')

Metadata file 22400512.xml saved
Metadata file 4310610.xml saved
Metadata file 13621365.xml saved


## Etwas einfacher: Arbeiten mit der Polymatheia Library

Genuin liefert die OAI-PMH-Schnittstelle XML-formatierte Metadaten gemäss verschiedenen Standards aus. Mit [Polymatheia](https://polymatheia.readthedocs.io/en/latest/index.html), einer spezifischen Python "Library" für OAI-Schnittstellen, können diese nicht nur **einfacher abgefragt** werden, sondern die Daten werden auch in einem **einfacher auswertbaren Format** ausgegeben. Das [Navigable Dictionary](https://polymatheia.readthedocs.io/en/latest/concepts.html) macht die direkte Addressierung einzelner Datenelemente möglich. Zudem kann es leicht im [JSON](https://de.wikipedia.org/wiki/JavaScript_Object_Notation)-Format gespeichert werden.

In [14]:
# Polymatheia library installieren
!pip install polymatheia

Defaulting to user installation because normal site-packages is not writeable


In [15]:
# verschiedene Module von Polymatheia importieren
from polymatheia.data.reader import OAISetReader               # list OAI sets
from polymatheia.data.reader import OAIMetadataFormatReader    # list available metadata formats
from polymatheia.data.reader import OAIRecordReader            # read metadata records from OAI
from polymatheia.data.writer import PandasDFWriter             # easy transformation of flat data into a dataframe
print("Successfully imported necessary libraries")

Successfully imported necessary libraries


In [16]:
# Basis-URL der OAI-Schnittstelle als definierte Variable macht die weitere Verwendung einfacher
oai = 'https://www.e-rara.ch/oai'

### Sets/Sammlungen und Metadatenformate abfragen

Polymatheia arbeitet mit vorgefertigten **Readern**, die einfach zur Abfrage gemäss der verschiedenen OAI verbs benutzt werden können. Der Reader nimmt die ausgelieferten Daten entgegen und speichert sie in einem Objekt (einem Navigable Dictionary), das dann ausgelesen werden kann.

In [17]:
# Der OAIMetadataFormatReader, der das OAI verb "ListMetadataFormats" nutzt
reader = OAIMetadataFormatReader(oai)
type(reader)

polymatheia.data.reader.OAIMetadataFormatReader

In [18]:
# Welche Metdatenformate sind verfügbar? Daten aus dem Reader auslesen
for formats in reader:
    #print(formats)
    print(formats.metadataPrefix)

oai_dc
mets
mods
rawmods
epicur


In [19]:
[formats.metadataPrefix for formats in reader]    # abgekürzte Variante der FOR-Schleife oben

['oai_dc', 'mets', 'mods', 'rawmods', 'epicur']

In [20]:
# Welche Sets/Sammlungen bietet die Schnittstelle an? Hierfür gibt es den OAISetReader.

reader = OAISetReader(oai)                         
[x for x in reader]

[{'setSpec': 'frc_g', 'setName': 'BCU Fribourg (GLN)'},
 {'setSpec': 'elibch', 'setName': 'Alle Bibliotheken'},
 {'setSpec': 'zbs', 'setName': 'Zentralbibliothek Solothurn'},
 {'setSpec': 'sbs', 'setName': 'Stadtbibliothek Schaffhausen'},
 {'setSpec': 'astrorara', 'setName': 'Astronomie-rara'},
 {'setSpec': 'bau_1', 'setName': 'UB Basel (DSV01)'},
 {'setSpec': 'kbg', 'setName': 'Kantonsbibliothek Graubünden'},
 {'setSpec': 'nep_r',
  'setName': 'Bibliothèque des Pasteurs, BPU Neuchâtel (RERO)'},
 {'setSpec': 'astrozut', 'setName': 'ETH-Bibliothek Zürich'},
 {'setSpec': 'stibi', 'setName': 'Stiftsbibliothek St. Gallen'},
 {'setSpec': 'frc_r', 'setName': 'BCU Fribourg (RERO)'},
 {'setSpec': 'ebs', 'setName': 'Eisenbibliothek Schlatt'},
 {'setSpec': 'nep_g',
  'setName': 'Bibliothèque des Pasteurs, BPU Neuchâtel (GLN)'},
 {'setSpec': 'lg1', 'setName': 'Biblioteca Salita dei Frati, Lugano'},
 {'setSpec': 'kbv', 'setName': 'Kantonsbibliothek Vadiana'},
 {'setSpec': 'demusmu', 'setName': 'De

In [21]:
# Mit dem PandasDFWriter von Polymatheia lassen sich die Set-Daten einfach in eine übersichtlichere Tabellenform überführen.

reader = OAISetReader(oai)
setspec = []                          # make an empty list named 'setspec'
for x in reader:
    setspec.append(x)                 # .append adds all the single reader-contents to the list 'setspec'
df = PandasDFWriter().write(setspec)  # write list 'setspec' into a Pandas dataframe named 'df'
df.style

,setSpec,setName
0,frc_g,BCU Fribourg (GLN)
1,elibch,Alle Bibliotheken
2,zbs,Zentralbibliothek Solothurn
3,sbs,Stadtbibliothek Schaffhausen
4,astrorara,Astronomie-rara
5,bau_1,UB Basel (DSV01)
6,kbg,Kantonsbibliothek Graubünden
7,nep_r,"Bibliothèque des Pasteurs, BPU Neuchâtel (RERO)"
8,astrozut,ETH-Bibliothek Zürich
9,stibi,Stiftsbibliothek St. Gallen


### Grösse eines Sets/einer Sammlung abrufen

Von Interesse ist oft auch die Grösse eines Sets/einer Sammlung. Diese ist etwas versteckt in den ausgebenen Daten, z.B. mit dem OAI verb `ListIdentifiers`, vorhanden. Um die Setgrösse einfach abzurufen, wird eine eine weitere Funktion `setsize()` definiert. Das entsprechende Set wird mit seinem Kurzzeichen abgefragt. 

In [22]:
# Wo steckt die Setgrösse?
IFrame('https://www.e-rara.ch/oai?verb=ListIdentifiers&set=vitruviana&metadataPrefix=oai_dc', width=820, height=300)

In [23]:
# Wie gross ist ein bestimtmes Set?

def setsize(Set):  
    '''
    Accesses the OAI interface and retrieves the size of a given OAI set.
    Parameters:
    Set: The 'setSpec' short cut of the desired OAI set.
    '''
    base_url = oai
    listsearch_term = {'verb': 'ListIdentifiers', 'metadataPrefix': 'oai_dc', 'set': Set}
    
    # Basic function
    def load_xml(params):
        '''
        Accesses the OAI interface according to given parameters and scrapes its content.
        Parameters:
        All available native OAI verbs and parameter/value pairs.
        '''
        response = requests.get(base_url, params=params)
        output_soup = soup(response.content, "lxml")
        return output_soup
    
    xml_soup = load_xml(listsearch_term)
    if xml_soup.resumptiontoken:
        set_size = int(xml_soup.resumptiontoken['completelistsize'])
    else:
        set_size = len(xml_soup.find_all('identifier'))
    return set_size

In [24]:
# alle Titel
setsize('')

92070

In [25]:
# "Gottfried Keller (1819-1890)"
setsize('pbgkeller')

641

### Auf ausgewählte Metadaten eines Sets zugreifen

Mit Polymatheia lassen sich leicht **massenhaft Metadaten** per OAI herunterladen. Hierbei können Metadatenstandard, Set/Sammlung und Anzahl von Datensätzen definiert werden. Der entsprechende Reader heisst `OAIRecordReader`. Die Daten sind wieder als Navigable Dictionary im Reader-Objekt gespeichert.

In [26]:
# x Metadatensätze eines Sets/einer Sammlung herunterladen und die "header section" der Datensätze auslesen

reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='stp', max_records=2)
for record in reader:
    print(record.header)

{
  "identifier": {
    "_text": "oai:www.e-rara.ch:25583780"
  },
  "datestamp": {
    "_text": "2021-05-20T06:38:03Z"
  },
  "setSpec": [
    {
      "_text": "stp"
    },
    {
      "_text": "book"
    },
    {
      "_text": "collections"
    },
    {
      "_text": "ch"
    },
    {
      "_text": "ch19"
    }
  ]
}
{
  "identifier": {
    "_text": "oai:www.e-rara.ch:25583895"
  },
  "datestamp": {
    "_text": "2021-05-26T06:08:47Z"
  },
  "setSpec": [
    {
      "_text": "stp"
    },
    {
      "_text": "book"
    },
    {
      "_text": "collections"
    },
    {
      "_text": "ch"
    },
    {
      "_text": "ch19"
    }
  ]
}


In [27]:
# Ebenso kann die "metadata section" des Datensätze ausgelesen werden - übersichtlicher in der abgekürzten Form der FOR-Schleife

[record.metadata for record in reader]

[{'{http://www.openarchives.org/OAI/2.0/oai_dc/}dc': {'_attrib': {'xsi_schemaLocation': 'http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd'},
   'dc_title': {'_text': 'Lesebuch für die Unterklassen schweizerischer Volksschulen'},
   'dc_creator': {'_text': 'Eberhard, Gerold'},
   'dc_subject': [{'_text': 'Deutschunterricht'}, {'_text': 'Volksschule'}],
   'dc_description': {'_text': 'von Gerold Eberhard'},
   'dc_publisher': {'_text': 'Schulthess'},
   'dc_contributor': {'_text': 'Eberhard, Gerold'},
   'dc_date': {'_text': '1881'},
   'dc_type': [{'_text': 'Text'}, {'_text': 'Book'}],
   'dc_format': {'_text': '3 Tle.'},
   'dc_identifier': [{'_text': 'doi:10.3931/e-rara-89249'},
    {'_text': 'https://www.e-rara.ch/stp/doi/10.3931/e-rara-89249'},
    {'_text': 'system:990095018120205521'}],
   'dc_relation': {'_text': 'vignette : https://www.e-rara.ch/stp/titlepage/doi/10.3931/e-rara-89249/128'},
   'dc_language': {'_text': 'ger'},
   'dc_righ

In [28]:
# Einzelne Metadatenfelder können im Navigable Dictionary direkt mit "Subsetting" oder per Punkt-Notation adressiert werden

for record in reader:
    print(record['header']['identifier']['_text'])       # Subsetting in Python
    print(record.header.identifier._text)                # Punkt-Notation des NavigableDict
    print('---')

oai:www.e-rara.ch:25583780
oai:www.e-rara.ch:25583780
---
oai:www.e-rara.ch:25583895
oai:www.e-rara.ch:25583895
---


In [29]:
# Für die Abfrage in der metadata section muss an einer Stelle allerdings zwingend die Subsetting-Syntax verwendet werden 
# wg. geschweiferter Klammern als Sonderzeichen

[record.metadata['{http://www.openarchives.org/OAI/2.0/oai_dc/}dc'].dc_title._text for record in reader]

['Lesebuch für die Unterklassen schweizerischer Volksschulen',
 'Deutsches Lesebuch für schweizerische Sekundar-, Real- und Bezirksschulen']

In [30]:
# Versuchen Sie weitere Metadaten-Elemente so herauszulesen!

[record.metadata['{http://www.openarchives.org/OAI/2.0/oai_dc/}dc']...... for record in reader]

SyntaxError: invalid syntax (2734541678.py, line 3)

In [31]:
# Metadatenfelder können auch mehrere Werte enthalten, wie die Angabe von mehreren Sets/Sammlungen, in die ein Titel gehört
# Die Daten werden dann als Liste ausgegeben, zu erkennen an den einfassenden eckigen Klammern.

for record in reader:
    print(record.header.setSpec)

[{'_text': 'stp'}, {'_text': 'book'}, {'_text': 'collections'}, {'_text': 'ch'}, {'_text': 'ch19'}]
[{'_text': 'stp'}, {'_text': 'book'}, {'_text': 'collections'}, {'_text': 'ch'}, {'_text': 'ch19'}]


In [32]:
# Wie beim Reader können die Listen-Elemente mit einer FOR-Schleife einzeln ausgelesen werden.
# D.h. wird haben dann 2 ineinandergeschachtelte FOR-Schleifen

for record in reader:
    for list_item in record.header.setSpec:
        print(list_item._text)
    print('---')

stp
book
collections
ch
ch19
---
stp
book
collections
ch
ch19
---


In [33]:
# Auch das "dc_subject"-Feld in der metadate section enthält eine Liste von mehreren Werten.
# Ergänzen Sie den Code, um die einzelnen Werte auszulesen!

for record in reader:
    for list_item in record.metadata['{http://www.openarchives.org/OAI/2.0/oai_dc/}dc']........:
        print(list_item.......)
    print('---')

SyntaxError: invalid syntax (1049347581.py, line 5)

Es gibt aber noch einen einfacheren Weg, an die einzelnen Metadaten-Elemente heranzuzukommen! Hier gibt es **keine Probleme mehr mit Einzelwerten versus Listen** als Datenwerte. Die einzelnen hierachisch angeordneten Metadaten-Elemente werden als Parameter eines spezifischen Befehls, `get`, gehandhabt.

Zu beachten ist dabei, dass im Falle von Listen ebenso eine Ergebnisliste (in eckigen Klammern) ausgegeben wird.


In [34]:
# Einfacher Zugriff auf alle Metadaten-Elemente mit dem "get"-Befehl

for record in reader:
    print(record.get(['header', 'identifier', '_text']))
    print(record.get(['header', 'setSpec', '_text']))
    print(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_title', '_text']))
    print(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_subject', '_text']))
    print('---')

oai:www.e-rara.ch:25583780
['stp', 'book', 'collections', 'ch', 'ch19']
Lesebuch für die Unterklassen schweizerischer Volksschulen
['Deutschunterricht', 'Volksschule']
---
oai:www.e-rara.ch:25583895
['stp', 'book', 'collections', 'ch', 'ch19']
Deutsches Lesebuch für schweizerische Sekundar-, Real- und Bezirksschulen
['Deutschunterricht', 'Volksschule']
---


### Ausgewählte Metadaten in Tabellenform bringen und als CSV abspeichern

In [35]:
setsize('ch19')

6285

In [36]:
reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='ch19', max_records=1000)

identifier = [] 
sets = []
creator = []
title = [] 
year = []
language = []
publisher = []
subjects = []
types = []
rights = []


for record in reader:
    identifier.append(record.get(['header', 'identifier', '_text']))
    sets.append(record.get(['header', 'setSpec', '_text']))
    creator.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_creator', '_text']))
    title.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_title', '_text']))
    year.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_date', '_text']))
    language.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_language', '_text']))
    publisher.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_publisher', '_text']))
    subjects.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_subject', '_text']))
    types.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_type', '_text']))
    rights.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_rights', '_text']))
    
df = pd.DataFrame(list(zip(identifier, title, creator, year, language, publisher, subjects, sets, types, rights)),
               columns =['identifier', 'title', 'creator', 'year', 'language', 'publisher', 'subjects', 'sets', 'types', 'rights'])
df

,identifier,title,creator,year,language,publisher,subjects,sets,types,rights
0,oai:www.e-rara.ch:14017,Funfzehn Ansichten der neuen St. Gotthards-Str...,"Lusser, Karl Franz",1833,ger,"bey Heinrich Füssli und Comp., Kunsthandlung z...","[ABBILDUNGEN UND BILDWERKE (DOKUMENTENTYP), GO...","[zut, book, collections, wihibe, ch, ch19]","[Text, Book]",pdm
1,oai:www.e-rara.ch:34616,Der Baugarten,"Esslinger, Christoph",1842,ger,Verlag von Hermann Trachsler,"[PANORAMENKARTEN, ZÜRICH, STADT (KANTON ZÜRICH)]","[zut, book, collections, wihibe, ch, ch19]","[Text, Book]",pdm
2,oai:www.e-rara.ch:34628,Der Höckler und das Schlösschen Maneck : ein ...,"Hardmeyer, Carl Wilhelm",1840,ger,bei Herrmann Trachsler,"[PANORAMENKARTEN, ZÜRICH, STADT (KANTON ZÜRICH)]","[zut, book, collections, wihibe, ch, ch19]","[Text, Book]",pdm
3,oai:www.e-rara.ch:366308,Ergebnisse der trigonometrischen Vermessungen ...,"Eschmann, Johannes",1840,ger,"Orell, Füssli","[AARBERG, ORT (KANTON BERN), BERN, STADT (KANT...","[zut, book, collections, wihibe, ch, ch19, pri...","[Text, Book]",pdm
4,oai:www.e-rara.ch:388892,"Allgemeines Künstlerlexikon, oder: Kurze Nachr...","Füssli, Johann Rudolf","[1806, 1821]",ger,"Orell, Füssli",None,"[zut, book, collections, wihibe, ch, ch19]","[Text, Book]",pdm
...,...,...,...,...,...,...,...,...,...,...
995,oai:www.e-rara.ch:6775826,Dissertation sur la partie de l'optique qui tr...,"La Rive, Auguste Arthur de",1823,fre,de l'imprimerie aux barrières,None,"[zut, book, collections, wihibe, ch, ch19]","[Text, Book]",pdm
996,oai:www.e-rara.ch:6776914,Schweizerische Handels-Akademie in Zürich : Pr...,"Bertsch, Ferdinand",1895,ger,Wildhaber,None,"[zuz, book, ch, ch19]","[Text, Book]",pdm
997,oai:www.e-rara.ch:6799599,Die Schweiz / 1 Der schweizerische Jura : se...,"Siegfried, Johann Jacob",1851,ger,"Druck und Verlag von Orell, Füssli & Comp.",None,"[zut, book, collections, wihibe, ch, ch19]","[Text, Book]",pdm
998,oai:www.e-rara.ch:6800516,Bericht der Finanz-Commission des Stadtrathes ...,Bern,1834,ger,Rätzer,None,"[bes_1, book, collections, bernensia, ch, ch19]","[Text, Book]",pdm


In [37]:
# Tabelle als CSV-Datei abspeichern, mit Semikolon als Spaltentrenner
outfile = 'e-rara_daten_semikolon.csv'
with open(outfile, mode='w', encoding='utf-8') as f:
    df.to_csv(f, index=False, sep=';')

In [38]:
# Tabelle als CSV-Datei abspeichern, mit Komma als Spaltentrenner (default)
outfile = 'e-rara_daten.csv'
with open(outfile, mode='w', encoding='utf-8') as f:
    df.to_csv(f, index=False)

## Ganze Metadatensätze als JSON-Dateien speichern

Daten, die als Navigable Dictionary vorliegen können einfach in **JSON-Formatierung** abgespeichert werden. Hierfür werden die einzelnen Datensätze aus dem Reader gelesen und **mit `.json` Dateiendung** mit dem **e-rara-Identifier als Dateiname** gespeichert. Im Beispiel wird vorab noch ein Ordner `json_data_dc` für die entstehenden Dateien angelegt.

In [39]:
# Auslesen der e-rara-Identifier eines Sets
reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='stp', max_records=10)

for record in reader:
    match = re.search('oai:www.e-rara.ch:(\d+)', record.header.identifier._text)
    ID = match.group(1)
    print(ID)

25583780
25583895
25584228
25584375
25584554
24724301
24724357
24724422
24724541
24724594


In [40]:
# Metadaten eines Sets/einer Sammlung als JSON-Dateien abspeichern, mit e-rara Identifiern als Dateiname
# Versuchen Sie auch das Gleiche im MODS-Standard und vergleichen Sie die abgespeicherten JSON-Dateien!

reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='stp', max_records=10)
os.makedirs('json_data_dc', exist_ok=True)
path = 'json_data_dc'

for record in reader:
    match = re.search('oai:www.e-rara.ch:(\d+)', record.header.identifier._text)
    ID = match.group(1)
    outfile = path + '/{}.json'.format(ID)
    try:
        with open(outfile, mode='w', encoding='utf-8') as f:
            f.write(str(record))
            print("Metadata file {}.json saved".format(ID))
    except:
            print("Saving metadata file failed".format(ID))
    finally:
            pass

Metadata file 25583780.json saved
Metadata file 25583895.json saved
Metadata file 25584228.json saved
Metadata file 25584375.json saved
Metadata file 25584554.json saved
Metadata file 24724301.json saved
Metadata file 24724357.json saved
Metadata file 24724422.json saved
Metadata file 24724541.json saved
Metadata file 24724594.json saved
